# Omgaan met ontbrekende gegevens
## None, math.nan, np.nan, pd.NA
Er zijn verschillende manieren om te definiëren dat gegevens ontbreken. Een Series met de verschillende waarden wordt gezien als een reeks objecten. 

In [ ]:
import math
import numpy as np
import pandas as pd
python_none = None
math_nan = math.nan
numpy_nan = np.nan
pandas_na = pd.NA
float_nan = float('nan')
ser = pd.Series([python_none, float_nan, math_nan, numpy_nan, pandas_na])
ser


## NumPy.nan is een floatwaarde
Kunnen we al die waarden niet omzetten naar float? Want we weten dat np.nan een float is. We gebruiken hier het nieuwe floattype van pandas.

In [ ]:
ser = pd.Series([python_none, float_nan, math_nan, numpy_nan, pandas_na], dtype=pd.Float64Dtype())
ser

## Uit nieuwsgierigheid: np.float64
Kunnen we ook het numpy float64 type gebruiken? Nee, dit geeft een fout. Het pandas datatype NA (NAType) wordt niet herkend als een numpy float. In pandas kunnen we dus best met Pandas datatypes werken. 

In [ ]:
ser = pd.Series([python_none, float_nan, math_nan, numpy_nan, pandas_na], dtype=np.float64)
ser

## None wordt herkend als pd.NA
Om te testen in Pandas of waarden 'leeg' (NA) zijn, kunnen we .isna() gebruiken

In [ ]:
ser = pd.Series([3.14, 2.5, None, 5])
ser.isna()

## Nan-waarden vergelijken.
Een nan-waarde (NA-waarde) is nooit gelijk aan een andere nan-waarde (NA-waarde). Net zoals in SQL gebruikt Python (en Pandas) three-valued-logic: iets kan waar zijn, iets kan niet waar zijn, of we weten het niet. (AI gebruikt two-valued-logic: iets is waar of niet waar, maar AI weet het altijd)

Het resultaat van deze test is een lege Series.

In [ ]:
ser = pd.Series([python_none, float_nan, math_nan, numpy_nan, pandas_na], dtype=pd.Float64Dtype())
ser[ser==pd.NA]

## pd.isna() en pd.notna()
Om te testen of iets NA is, moeten we een functie gebruiken. De omgekeerde test gebeurt met pd.notna()

In [ ]:
ser = pd.Series([python_none, float_nan, math_nan, numpy_nan, pandas_na, 1.23], dtype=pd.Float64Dtype())
print(ser.isna())
print(ser.notna())
print(ser.notnull()) #alias voor notna()

## Nan-waarden voor integers
In NumPy kunnen integers niet 'leeg' zijn. Met het nieuwere Pandas type kan dat wel.

In [ ]:
ser_object=pd.Series([1, 2, 3, pd.NA])
print("klassieke integers")
ser_object.info()
ser_int = pd.Series([1, 2, 3, pd.NA], dtype=pd.Int64Dtype())
print("\nPandas integers")
ser_int.info()

## Pandas strings kunnen ook 'leeg' zijn. 
Het nieuwere Pandas stringtype kan ook 'leeg' zijn

In [ ]:
ser_object = pd.Series(['Karen', 'Kristel', 'Kathleen', None])
print('klassieke strings')
ser_object.info()
ser_string = pd.Series(['Karen', 'Kristel', 'Kathleen', None], dtype=pd.StringDtype())
print('\nPandas strings')
ser_string.info()

## NA-waarden zijn meestal niet zo interessant
Daarom is er een functie waarmee we NA-gegevens kunnen verwijderen. 

In [ ]:
ser = pd.Series([pd.NA, 1, 2, np.nan, None], dtype=pd.Int64Dtype())
ser.dropna()

## En hoe zit het met NA in een dataframe?
Wanneer we met een dataframe werken, kunnen we nooit 1 cel verwijderen. De functie .dropna() heeft een axis parameter die standaard gelijk is aan 0 (of 'index')

In [ ]:
df = pd.DataFrame({'voornaam':['Karen', 'Kristel', 'Kathleen'], 'achternaam': ['Damen', 'Verbeke', pd.NA]})
df.dropna() #hetzelfde als df.dropna(axis='index') of df.dropna(axis=0)

## Een kolom met NA-waarden verwijderen
Door een axis=1 (of axis='columns') argument mee te geven, kunnen we een kolom verwijderen.

In [ ]:
df = pd.DataFrame({'voornaam':['Karen', 'Kristel', 'Kathleen'], 'achternaam': ['Damen', 'Verbeke', pd.NA]})
df.dropna(axis='columns') #of df.dropna(axis=1)

## NA-waarden vervangen
Soms kunnen we NA-waarden vervangen door een vaste waarde (bijvoorbeeld het gemiddelde). In het volgende voorbeeld moeten we een float-type kiezen omdat het gemiddelde natuurlijk een float zal zijn.

Let op: in pandas is er geen probleem met NA-waarden wanneer we het gemiddelde willen berekenen. (er bestaat geen pd.nanmean())

In [ ]:
ser = pd.Series([1, 6, 9, 3, pd.NA, 8], dtype=pd.Float64Dtype())
gemiddelde = ser.mean()
ser.fillna(gemiddelde)

## Waarden van de vorige cel herhalen
Soms is de waarde van een naburige cel een goede benadering van een ontbrekende waarde. Denk bijvoorbeeld aan waarden van een aandeel op de beurs. Wanneer de prijs voor een bepaalde dag ontbreekt, kunnen we er misschien vanuit gaan dat de waarde niet veranderd is ten opzichte van de vorige dag. In het onderstaande voorbeeld wordt de waarde 116 herhaald

In [ ]:
rng = np.random.default_rng(42)
datums = pd.date_range(pd.to_datetime('2026-01-02'), pd.to_datetime('2026-01-06'))
waarden = rng.integers(100, 121, len(datums))
print(waarden)
ser = pd.Series(waarden, index=datums, dtype=pd.Int64Dtype())
ser.iat[2] = pd.NA
display(ser)
ser.ffill()

## Waarde van de volgende cel herhalen
Wanneer de eerste waarde ontbreekt, werkt dit natuurlijk niet. Er bestaat echter ook een .bfill() functie

In [ ]:
rng = np.random.default_rng(42)
datums = pd.date_range(pd.to_datetime('2026-01-02'), pd.to_datetime('2026-01-06'))
waarden = rng.integers(100, 121, len(datums))
print(waarden)
ser = pd.Series(waarden, index=datums, dtype=pd.Int64Dtype())
ser.iat[0] = pd.NA
display(ser)
ser.bfill()

## Interpolate
De functie .ffill() en .bfill() zijn misschien niet de beste oplossing wanneer we zowel de vorige als de volgende waarde kennen. Misschien dat een interpolatie dan beter is.  (.interpolate() heeft meteen ook het type veranderd naar een Float64Dtype)

In [ ]:
rng = np.random.default_rng(42)
datums = pd.date_range(pd.to_datetime('2026-01-02'), pd.to_datetime('2026-01-06'))
waarden = rng.integers(100, 121, len(datums))
print(waarden)
ser = pd.Series(waarden, index=datums, dtype=pd.Int64Dtype())
ser.iat[2] = pd.NA
display(ser)
ser.interpolate()

## Interpolatie op basis van de index
Wanneer er een aantal datums ontbreken in de index, is een lineaire interpolatie misschien niet de beste oplossing. We weten immers niet wat de waarde van de volgende dag was. In het volgende voorbeeld ontbreken de gegevens van 6 en 7 januari. De geïnterpoleerde waarde ligt daardoor dichter bij de waarde van 5 januari.

In [ ]:
rng = np.random.default_rng(42)
datums1 = pd.date_range(pd.to_datetime('2026-01-02'), pd.to_datetime('2026-01-05'))
datums2 = pd.date_range(pd.to_datetime('2026-01-08'), pd.to_datetime('2026-01-11'))
datums = datums1.union(datums2)
waarden = rng.integers(100, 121, len(datums))

ser = pd.Series(waarden, index=datums, dtype=pd.Int64Dtype())
ser.iat[3] = pd.NA
print('lineaire interpolatie')
display(ser.interpolate())
print('\nInterpolatie rekening houdend met datums')
display(ser.interpolate('index'))